In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

from wordcloud import WordCloud

from sklearn.model_selection import KFold, train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [8]:
#training = pd.read_csv('C:/Users/dlsim/Documents/_College/1 UC Davis/4 Senior Year/Win - STA 142a - Statistical Learning I/ipynb files/142 Final Project/train.csv')
#test = pd.read_csv('C:/Users/dlsim/Documents/_College/1 UC Davis/4 Senior Year/Win - STA 142a - Statistical Learning I/ipynb files/142 Final Project/test.csv')
#test_labels = pd.read_csv('C:/Users/dlsim/Documents/_College/1 UC Davis/4 Senior Year/Win - STA 142a - Statistical Learning I/ipynb files/142 Final Project/test_labels.csv')

In [9]:
#training = pd.read_csv("C:/Users/Falak/Desktop/NLP/train.csv")
#test = pd.read_csv("C:/Users/Falak/Desktop/NLP/test.csv")
#test_labels = pd.read_csv("C:/Users/Falak/Desktop/NLP/test_labels.csv")

In [2]:
# Read data
training = pd.read_csv('C:/Users/Alvis/STA142A/train.csv')
test = pd.read_csv('C:/Users/Alvis/STA142A/test.csv')
test_labels = pd.read_csv("C:/Users/Alvis/STA142A/test_labels.csv")

In [3]:
### Cleaning test data ######

# Getting rid of the rows containing -1s in the test_labels dataset
cleaned_testlabels = test_labels[test_labels != -1]
cleaned_testlabels = cleaned_testlabels.dropna()
cleaned_testlabels.head(20)

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
5,0001ea8717f6de06,0.0,0.0,0.0,0.0,0.0,0.0
7,000247e83dcc1211,0.0,0.0,0.0,0.0,0.0,0.0
11,0002f87b16116a7f,0.0,0.0,0.0,0.0,0.0,0.0
13,0003e1cccfd5a40a,0.0,0.0,0.0,0.0,0.0,0.0
14,00059ace3e3e9a53,0.0,0.0,0.0,0.0,0.0,0.0
16,000663aff0fffc80,0.0,0.0,0.0,0.0,0.0,0.0
17,000689dd34e20979,0.0,0.0,0.0,0.0,0.0,0.0
19,000844b52dee5f3f,0.0,0.0,0.0,0.0,0.0,0.0
21,00091c35fa9d0465,1.0,0.0,0.0,0.0,0.0,0.0
22,000968ce11f5ee34,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
#converts all values to int
cleaned_testlabels.apply(pd.to_numeric, downcast = 'integer', errors = 'ignore')

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
5,0001ea8717f6de06,0,0,0,0,0,0
7,000247e83dcc1211,0,0,0,0,0,0
11,0002f87b16116a7f,0,0,0,0,0,0
13,0003e1cccfd5a40a,0,0,0,0,0,0
14,00059ace3e3e9a53,0,0,0,0,0,0
...,...,...,...,...,...,...,...
153150,fff8f64043129fa2,0,0,0,0,0,0
153151,fff9d70fe0722906,0,0,0,0,0,0
153154,fffa8a11c4378854,0,0,0,0,0,0
153155,fffac2a094c8e0e2,1,0,1,0,1,0


In [5]:
# Joining the test_label dataset to the test dataset
test_with_comments = cleaned_testlabels.join(test.set_index('id'), on='id')

test_with_comments

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate,comment_text
5,0001ea8717f6de06,0.0,0.0,0.0,0.0,0.0,0.0,Thank you for understanding. I think very high...
7,000247e83dcc1211,0.0,0.0,0.0,0.0,0.0,0.0,:Dear god this site is horrible.
11,0002f87b16116a7f,0.0,0.0,0.0,0.0,0.0,0.0,"""::: Somebody will invariably try to add Relig..."
13,0003e1cccfd5a40a,0.0,0.0,0.0,0.0,0.0,0.0,""" \n\n It says it right there that it IS a typ..."
14,00059ace3e3e9a53,0.0,0.0,0.0,0.0,0.0,0.0,""" \n\n == Before adding a new product to the l..."
...,...,...,...,...,...,...,...,...
153150,fff8f64043129fa2,0.0,0.0,0.0,0.0,0.0,0.0,":Jerome, I see you never got around to this…! ..."
153151,fff9d70fe0722906,0.0,0.0,0.0,0.0,0.0,0.0,==Lucky bastard== \n http://wikimediafoundatio...
153154,fffa8a11c4378854,0.0,0.0,0.0,0.0,0.0,0.0,==shame on you all!!!== \n\n You want to speak...
153155,fffac2a094c8e0e2,1.0,0.0,1.0,0.0,1.0,0.0,MEL GIBSON IS A NAZI BITCH WHO MAKES SHITTY MO...


In [6]:
# Preprocessing
import re
from nltk.corpus import stopwords
stop = stopwords.words('english')

def preprocess(training, test):
    training['clean'] = training.apply(lambda row: row['comment_text'].replace("\n"," "), axis=1) #removes new line character
    test['clean'] = test.apply(lambda row: row['comment_text'].replace("\n"," "), axis=1)

    #removes urls
    training['clean'] = training.apply(lambda row: re.sub('http://\S+|https://\S+', '', row['clean']), axis=1)
    test['clean'] = test.apply(lambda row: re.sub('http://\S+|https://\S+', '',row['clean']), axis=1)

    #remove all non-alphanumeric values
    training['clean'] = training.apply(lambda row: re.sub('[^A-Za-z0-9\' ]+', '',row['clean']), axis=1)
    test['clean'] = test.apply(lambda row: re.sub('[^A-Za-z0-9\' ]+', '',row['clean']), axis=1)

    #remove stopwords as they occupy major chunk of the vocabulary
    training['clean'] = training['clean'].apply(lambda x: ' '.join([word for word in x.split() if not word in (stop)]))
    test['clean'] = test['clean'].apply(lambda x: ' '.join([word for word in x.split() if not word in (stop)]))

    #removes all additional spaces
    training['clean'] = training.apply(lambda row: re.sub(' +', ' ', row['clean']).strip(), axis=1)
    test['clean'] = test.apply(lambda row: re.sub(' +', ' ', row['clean']).strip(), axis=1)
    
    #lowercase end result
    training['clean'] = training['clean'].str.lower()
    test = test['clean'].str.lower()

preprocess(training, test_with_comments)
training.head(10)
test.head(10)

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.
5,0001ea8717f6de06,Thank you for understanding. I think very high...
6,00024115d4cbde0f,Please do not add nonsense to Wikipedia. Such ...
7,000247e83dcc1211,:Dear god this site is horrible.
8,00025358d4737918,""" \n Only a fool can believe in such numbers. ..."
9,00026d1092fe71cc,== Double Redirects == \n\n When fixing double...


In [17]:
### Checking accuracy scores for each label - Logistic Regression
for i in range(1,7):
    X_train = training
    X_test = test_with_comments
    Y_train = X_train.iloc[:,i+1]

    tfidf_vectorizer=TfidfVectorizer(use_idf=True, max_df=0.7) #Vectorization of training cleaned comments
    tfidf_vectorizer.fit_transform(X_train['clean'].values)

    train_feature_set=tfidf_vectorizer.transform(X_train['clean'].values)
    test_feature_set=tfidf_vectorizer.transform(X_test['clean'].values)

    scikit_log_reg = LogisticRegression(verbose=1, solver='liblinear',random_state=0, C=5, penalty='l2',max_iter=1000)
    model = scikit_log_reg.fit(train_feature_set, Y_train) #Model build using training data cleaned comments

    Y_predicted = model.predict(test_feature_set) #Y_predicted are the predicted labels from our model
    Y_predicted_prob = model.predict_proba(test_feature_set) #proba model shows two columns: 1st column is prob that it is classified as 0, 2nd column is prob that it is classified as 1
    #print(proba)
 
    #Predicted Accuracy score

    Y_test = X_test.iloc[:,i]
    print(f"The Logistic Regression classification accuracy of the fitted model for the {test_with_comments.columns[i]} label is {accuracy_score(Y_test, Y_predicted)}")

[LibLinear]The classification accuracy of the fitted model for the toxic label is 0.9299759292256713
[LibLinear]The classification accuracy of the fitted model for the severe_toxic label is 0.9931382662790335
[LibLinear]The classification accuracy of the fitted model for the obscene label is 0.9653474631904717
[LibLinear]The classification accuracy of the fitted model for the threat label is 0.9967332520553941
[LibLinear]The classification accuracy of the fitted model for the insult label is 0.9636906436587577
[LibLinear]The classification accuracy of the fitted model for the identity_hate label is 0.9902779080308857


In [12]:
#Probability Model for each label - Logistic Regression
scikit_log_reg = LogisticRegression(verbose=1, solver='liblinear',random_state=0, C=5, penalty='l2',max_iter=1000) 

import sys, os
def LogRegFit_NoMessage(X, Y, logreg_obj):
    old_stdout = sys.stdout
    sys.stdout = open(os.devnull, 'w')
    
    model = logreg_obj.fit(X, Y) #Model build using training data cleaned comments
    
    sys.stdout = old_stdout
    
    return(model)
def calcClassProba(X_train, X_test, Class):    
    Y_train_label = X_train[Class]
    
    tfidf_vectorizer=TfidfVectorizer(use_idf=True, max_df=0.7) #Vectorization of training cleaned comments
    tfidf_vectorizer.fit_transform(X_train['clean'].values)
    
    train_features=tfidf_vectorizer.transform(X_train['clean'].values)
    test_features =tfidf_vectorizer.transform(X_test['clean'].values)
        
    model = LogRegFit_NoMessage(train_features, Y_train_label, scikit_log_reg)
    
    # probability for the given labels using X test set
    Y_prob = model.predict_proba(test_features)

    return(Y_prob[:,1])

classes = ["toxic","severe_toxic","obscene","threat","insult","identity_hate"]

prob_list = {"toxic":[],"severe_toxic":[],"obscene":[],"threat":[],"insult":[],"identity_hate":[]}

for c in classes:
    prob_list[c].append(calcClassProba(training, test_with_comments, c))
    
prob_list

{'toxic': [array([0.00371341, 0.64768016, 0.09695487, ..., 0.7974892 , 0.99913455,
         0.05323492])],
 'severe_toxic': [array([0.00036812, 0.0023743 , 0.00103369, ..., 0.01244504, 0.12633439,
         0.00294341])],
 'obscene': [array([0.00184688, 0.04232193, 0.02236229, ..., 0.04993189, 0.99661367,
         0.0052893 ])],
 'threat': [array([0.00013652, 0.00253933, 0.00094628, ..., 0.00258464, 0.00210866,
         0.00086979])],
 'insult': [array([0.0052016 , 0.14908443, 0.01701469, ..., 0.17228261, 0.96434512,
         0.01677638])],
 'identity_hate': [array([0.00052716, 0.00211506, 0.00464   , ..., 0.11144712, 0.09737732,
         0.00321977])]}

In [19]:
### Checking accuracy scores for each label - KNN (for 20000 samples; takes about 10 minutes)
for i in range(1,7):
    X_train = training
    X_test = test_with_comments
    Y_train = X_train.iloc[:,i+1].sample(20000)

    tfidf_vectorizer=TfidfVectorizer(use_idf=True, max_df=0.7) #Vectorization of training cleaned comments
    tfidf_vectorizer.fit_transform(X_train['clean'].values)

    train_feature_set=tfidf_vectorizer.transform(np.random.choice(training['clean'].values, 20000))
    test_feature_set=tfidf_vectorizer.transform(X_test['clean'].values)

    scikit_knn = KNeighborsClassifier(n_neighbors = 5)
    model = scikit_knn.fit(train_feature_set, Y_train) #Model build using training data cleaned comments

    Y_predicted = model.predict(test_feature_set) #Y_predicted are the predicted labels from our model
    Y_predicted_prob = model.predict_proba(test_feature_set) #proba model shows two columns: 1st column is prob that it is classified as 0, 2nd column is prob that it is classified as 1
    #print(proba)
 
    #Predicted Accuracy score

    Y_test = X_test.iloc[:,i]
    print(f"The KNN classification accuracy of the fitted model for the {test_with_comments.columns[i]} label is {accuracy_score(Y_test, Y_predicted)}")

The KNN classification accuracy of the fitted model for the toxic label is 0.9048110287911469
The KNN classification accuracy of the fitted model for the severe_toxic label is 0.9942636531307637
The KNN classification accuracy of the fitted model for the obscene label is 0.9399012160430148
The KNN classification accuracy of the fitted model for the threat label is 0.9967019913095126
The KNN classification accuracy of the fitted model for the insult label is 0.9463409296945825
The KNN classification accuracy of the fitted model for the identity_hate label is 0.9888711744662227


In [26]:
#Probability Model for each label - kNN (sampled 20000. Note that because kNN=5, our probabilites are in increments of .2)
scikit_knn = KNeighborsClassifier(n_neighbors = 5)

import sys, os
def kNNFit_NoMessage(X, Y, knn_obj):
    old_stdout = sys.stdout
    sys.stdout = open(os.devnull, 'w')
    
    model = knn_obj.fit(X, Y) #Model build using training data cleaned comments
    
    sys.stdout = old_stdout
    
    return(model)

def calckNNClassProba(X_train, X_test, Class):    
    Y_train_label = X_train[Class].sample(20000)
    
    tfidf_vectorizer=TfidfVectorizer(use_idf=True, max_df=0.7) #Vectorization of training cleaned comments
    tfidf_vectorizer.fit_transform(X_train['clean'].values)
    
    train_features=tfidf_vectorizer.transform(np.random.choice(X_train['clean'].values, 20000))
    test_features =tfidf_vectorizer.transform(np.random.choice(X_test['clean'].values, 20000))
        
    model = kNNFit_NoMessage(train_features, Y_train_label, scikit_knn)
    
    # probability for the given labels using X test set
    Y_prob = model.predict_proba(test_features)

    return(Y_prob[:,1])

classes = ["toxic","severe_toxic","obscene","threat","insult","identity_hate"]

prob_list = {"toxic":[],"severe_toxic":[],"obscene":[],"threat":[],"insult":[],"identity_hate":[]}

for c in classes:
    prob_list[c].append(calckNNClassProba(training, test_with_comments, c))
    
prob_list

{'toxic': [array([0. , 0.4, 0.2, ..., 0.2, 0. , 0. ])],
 'severe_toxic': [array([0., 0., 0., ..., 0., 0., 0.])],
 'obscene': [array([0., 0., 0., ..., 0., 0., 0.])],
 'threat': [array([0., 0., 0., ..., 0., 0., 0.])],
 'insult': [array([0., 0., 0., ..., 0., 0., 0.])],
 'identity_hate': [array([0., 0., 0., ..., 0., 0., 0.])]}

In [7]:
### Checking accuracy scores for each label - SVM
for i in range(1,7):
    X_train = training
    X_test = test_with_comments
    Y_train = X_train.iloc[:,i+1].sample(1000)

    tfidf_vectorizer=TfidfVectorizer(use_idf=True, max_df=0.7) #Vectorization of training cleaned comments
    tfidf_vectorizer.fit_transform(X_train['clean'].values)

    train_feature_set=tfidf_vectorizer.transform(np.random.choice(training['clean'].values, 1000))
    test_feature_set=tfidf_vectorizer.transform(X_test['clean'].values)

    scikit_svm = SVC(kernel = "linear", C = 0.25, random_state = 999, class_weight = 'balanced')
    model = scikit_svm.fit(train_feature_set, Y_train) #Model build using training data cleaned comments

    Y_predicted = model.predict(test_feature_set) #Y_predicted are the predicted labels from our model
 
    #Predicted Accuracy score

    Y_test = X_test.iloc[:,i]
    print(f"The SVM classification accuracy of the fitted model for the {test_with_comments.columns[i]} label is {accuracy_score(Y_test, Y_predicted)}")

The SVM classification accuracy of the fitted model for the toxic label is 0.8878677045234299
The SVM classification accuracy of the fitted model for the severe_toxic label is 0.9941229797742974
The SVM classification accuracy of the fitted model for the obscene label is 0.9389790240395136
The SVM classification accuracy of the fitted model for the threat label is 0.9967019913095126
The SVM classification accuracy of the fitted model for the insult label is 0.9438088092781893
The SVM classification accuracy of the fitted model for the identity_hate label is 0.9868392259839319


In [9]:
#Probability Model for each label - SVM (sampled 10000)
scikit_svm = SVC(kernel = "linear", C = 0.25, random_state = 999, probability = True, class_weight = 'balanced')

import sys, os
def svmFit_NoMessage(X, Y, svm_obj):
    old_stdout = sys.stdout
    sys.stdout = open(os.devnull, 'w')
    
    model = svm_obj.fit(X, Y) #Model build using training data cleaned comments
    
    sys.stdout = old_stdout
    
    return(model)

def calcSVMClassProba(X_train, X_test, Class):    
    Y_train_label = X_train[Class].sample(1000)
    
    tfidf_vectorizer=TfidfVectorizer(use_idf=True, max_df=0.7) #Vectorization of training cleaned comments
    tfidf_vectorizer.fit_transform(X_train['clean'].values)
    
    train_features=tfidf_vectorizer.transform(np.random.choice(X_train['clean'].values, 1000))
    test_features =tfidf_vectorizer.transform(np.random.choice(X_test['clean'].values, 1000))
        
    model = svmFit_NoMessage(train_features, Y_train_label, scikit_svm)
    
    # probability for the given labels using X test set
    Y_prob = model.predict_proba(test_features)

    return(Y_prob[:,1])

classes = ["toxic","severe_toxic","obscene","threat","insult","identity_hate"]

prob_list = {"toxic":[],"severe_toxic":[],"obscene":[],"threat":[],"insult":[],"identity_hate":[]}

for c in classes:
    prob_list[c].append(calcSVMClassProba(training, test_with_comments, c))
    
prob_list

{'toxic': [array([0.08154221, 0.09204253, 0.10343115, 0.08539819, 0.09816913,
         0.13649662, 0.05918495, 0.08280487, 0.07860191, 0.05975398,
         0.0933439 , 0.09995618, 0.12472788, 0.07891222, 0.09409766,
         0.07343455, 0.08922894, 0.10465691, 0.06902625, 0.08509705,
         0.08415008, 0.10753699, 0.07938593, 0.09720458, 0.08890868,
         0.08518839, 0.08019521, 0.08537781, 0.09101979, 0.08913761,
         0.08869887, 0.09515059, 0.08379276, 0.07654767, 0.13367274,
         0.11996336, 0.06616033, 0.09722583, 0.11394999, 0.09113621,
         0.11143051, 0.08237157, 0.06957105, 0.10506645, 0.08903854,
         0.0931293 , 0.06859439, 0.07696643, 0.08921401, 0.07494026,
         0.10693479, 0.06957164, 0.08456872, 0.07596389, 0.08217757,
         0.0983923 , 0.09402363, 0.08141978, 0.09982181, 0.08751333,
         0.08234867, 0.07437921, 0.12320584, 0.11594288, 0.15031023,
         0.06221015, 0.11655549, 0.08703117, 0.08957293, 0.0826228 ,
         0.09937235, 0.08

In [13]:
# outputting probability table
a = test_with_comments 
a["toxic.p"] = prob_list["toxic"][0]

#assign probabilities to each class
for c in classes:
    test_with_comments[c] = prob_list[c][0]
test_with_comments

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate,comment_text,clean
5,0001ea8717f6de06,0.003713,0.000368,0.001847,0.000137,0.005201,0.000527,Thank you for understanding. I think very high...,Thank understanding I think highly would rever...
7,000247e83dcc1211,0.647680,0.002374,0.042322,0.002539,0.149250,0.002115,:Dear god this site is horrible.,Dear god site horrible
11,0002f87b16116a7f,0.096955,0.001034,0.022362,0.000946,0.017017,0.004640,"""::: Somebody will invariably try to add Relig...",Somebody invariably try add Religion Really Yo...
13,0003e1cccfd5a40a,0.002696,0.001045,0.005064,0.000287,0.001463,0.000946,""" \n\n It says it right there that it IS a typ...",It says right IS type The Type institution nee...
14,00059ace3e3e9a53,0.002601,0.000105,0.001586,0.000115,0.000924,0.000051,""" \n\n == Before adding a new product to the l...",Before adding new product list make sure relev...
...,...,...,...,...,...,...,...,...,...
153150,fff8f64043129fa2,0.003942,0.000348,0.001587,0.000399,0.001041,0.000305,":Jerome, I see you never got around to this…! ...",Jerome I see never got around Im surprised I l...
153151,fff9d70fe0722906,0.479290,0.022995,0.478501,0.001635,0.440722,0.011160,==Lucky bastard== \n http://wikimediafoundatio...,Lucky bastard Heh famous I kida envy Congrats
153154,fffa8a11c4378854,0.797489,0.012445,0.049932,0.002585,0.172575,0.111447,==shame on you all!!!== \n\n You want to speak...,shame You want speak gays romanians
153155,fffac2a094c8e0e2,0.999135,0.126334,0.996614,0.002109,0.964338,0.097377,MEL GIBSON IS A NAZI BITCH WHO MAKES SHITTY MO...,MEL GIBSON IS A NAZI BITCH WHO MAKES SHITTY MO...
